In [16]:
import pandas as pd
import numpy as np
import keras
from keras import layers

In [28]:
data = pd.read_csv(r'C:\Users\USER\Downloads\mnist.csv')
data

,id,class,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,31953,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34452,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,60897,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,36953,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1981,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,25268,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3996,6473,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3997,5821,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3998,1751,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
num_classes = 10
input_shape = (28, 28, 1)

# Extract features and labels
x_data = data.iloc[:, 2:].values  # Pixel values start from the third column
y_data = data.iloc[:, 1].values   # Labels are in the second column

# Normalize the pixel values
x_data = x_data.astype("float32") / 255

# Reshape the data
x_data = x_data.reshape(-1, 28, 28, 1)

# Split into training and test sets (assuming a 80-20 split)
split_index = int(.8 * len(x_data))
x_train, x_test = x_data[:split_index], x_data[split_index:]
y_train, y_test = y_data[:split_index], y_data[split_index:]

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Convert labels to categorical format
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (3200, 28, 28, 1)
3200 train samples
800 test samples


In [51]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

# Display model summary
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=128, epochs=15, validation_split=0.2)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

Epoch 1/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 119ms/step - accuracy: 0.2070 - loss: 2.1857 - val_accuracy: 0.7516 - val_loss: 1.5590
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.6596 - loss: 1.3246 - val_accuracy: 0.8594 - val_loss: 0.5615
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.8005 - loss: 0.6275 - val_accuracy: 0.8906 - val_loss: 0.3638
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.8803 - loss: 0.4031 - val_accuracy: 0.9203 - val_loss: 0.2662
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.9001 - loss: 0.3152 - val_accuracy: 0.9391 - val_loss: 0.2182
Epoch 6/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.9215 - loss: 0.2561 - val_accuracy: 0.9391 - val_loss: 0.1920
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.9361 - loss: 0.2086 - val_accuracy: 0.9484 - val_loss: 0.1729
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.9353 - loss: 0.2064 - val_accuracy: 0.9500 - 

In [49]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.13694390654563904
Test accuracy: 0.9637500047683716
